<a href="https://colab.research.google.com/github/ChiaHouTan/SmartTechCa2/blob/main/D00223937_Smart_Tech_CA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ChiaHouTan/SmartTechCa2.git

Cloning into 'smarttechca1'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import pickle
import pandas as pd
import random
import cv2
import requests
from PIL import Image

#git add .
#git status
#git commit -m "First Commit" then after finish everything do below
#git branch -M main
#git push -uf origin main